In [22]:
import numpy as np
import IPython.display as ipd
import math
import pandas as pd

In [23]:
#reading file
data = pd.read_csv(r"C:\Uni\ML\Bach\F.txt", delimiter = "\t", names = ["V1", "V2", "V3", "V4"])
voices = ["V1", "V2", "V3", "V4"]

In [24]:
#uniq_pitchn stores all the unique pitches in a voice.
#Vn_dict stores a dictionary which maps each pitch to a continuous range of indexes
uniq_pitch1=data["V1"].unique()
V1_dict={k: v for v, k in enumerate(uniq_pitch1)}
uniq_pitch2=data["V2"].unique()
V2_dict={k: v for v, k in enumerate(uniq_pitch2)}
uniq_pitch3=data["V3"].unique()
V3_dict={k: v for v, k in enumerate(uniq_pitch3)}

uniq_pitch4=data["V4"].unique()
V4_dict={k: v for v, k in enumerate(uniq_pitch4)}

In [25]:
#The transition matrix gives probabilities of transition from one hidden state to the next
def Transition_matrix(v,pitch_dict):
    Transition=np.zeros([len(pitch_dict),len(pitch_dict)]) #create empty transition matrix
    for index,pitch in enumerate(data[v]):
        if index<(len(data[v])-1):
            #Transition[i][j] indicates the probability of transition from i to j. Every time this transition is encountered, incremet by 1
            Transition[pitch_dict[pitch]][pitch_dict[data[v][index+1]]]=Transition[pitch_dict[pitch]][pitch_dict[data[v][index+1]]]+1
    #convert the counts to probabilities
    for i in range(0,len(Transition)):
        s=sum(Transition[i])
        for j in range(0,len(Transition[i])):
            Transition[i][j]=Transition[i][j]/s
    return Transition

In [27]:
Transition_V1=Transition_matrix("V1",V1_dict)
Transition_V1=Transition_V1.tolist()

Transition_V2=Transition_matrix("V2",V2_dict)
Transition_V2=Transition_V2.tolist()

Transition_V3=Transition_matrix("V3",V3_dict)
Transition_V3=Transition_V3.tolist()

#a unigram window matrix for v4 is created for special case as explained below
prob_V4=Transition_matrix("V4",V4_dict)
prob_V4=prob_V4.tolist()

In [28]:
uniq_pitch4=data["V4"].unique()
#get all unique chords, i.e combination of V1,V2,V3
uniq_rows=data.drop(["V4"],axis=1).drop_duplicates()
chord_list=uniq_rows.values.tolist()

In [29]:
#create the emission matrix which gives probability of an observable based on the hidden state.
Emission=np.zeros([len(chord_list),len(uniq_pitch4)])
for i,row in data.iterrows():
    #Emission[i][j] gives the probability of hidden state i producing an observable j.Every time this emission is encountered, incremet by 1
    Emission[chord_list.index([row["V1"],row["V2"],row["V3"]])][V4_dict[row["V4"]]]=Emission[chord_list.index([row["V1"],row["V2"],row["V3"]])][V4_dict[row["V4"]]]+1

In [30]:
#convert counts to probabilities
for i in range(0,len(Emission)):
    Emission[i]=Emission[i]/sum(Emission[i])

In [31]:
Emission=Emission.tolist()

In [61]:
count=1
exe=0
data_pred=pd.DataFrame(columns=voices)
#store the last row pitches of the dataset
prev_p1=data["V1"][len(data)-1]
prev_p2=data["V2"][len(data)-1]
prev_p3=data["V3"][len(data)-1]
prev_p4=data["V4"][len(data)-1]

chord_list_str=[str(i) for i in chord_list]
while(count<300):
    #predict the hidden states
    hidden_p1=int(np.random.choice(uniq_pitch1,1,p=Transition_V1[V1_dict[int(prev_p1)]]))
    hidden_p2=int(np.random.choice(uniq_pitch2,1,p=Transition_V2[V2_dict[int(prev_p2)]]))
    hidden_p3=int(np.random.choice(uniq_pitch3,1,p=Transition_V3[V3_dict[int(prev_p3)]]))
    #the hidden states are combined to form a "chord"
    chords=[hidden_p1,hidden_p1,hidden_p1]
    
    if chords not in chord_list:
        #if there is not emission probability for the chord, V4 is predicted as in a unigram markov chain.exe incremented with every such pseudo random step
        exe=exe+1
        obs_p4=int(np.random.choice(uniq_pitch4,1,p=prob_V4[V4_dict[int(prev_p4)]]))
    else:
        #if chord exists, V4 is the observable produced by the hidden state chord
        obs_p4=int(np.random.choice(uniq_pitch4,1,p=Emission[chord_list.index(chords)]))
    
    #add the prediction
    data_pred.loc[len(data_pred)]=[hidden_p1,hidden_p2,hidden_p3,obs_p4]
    count=count+1
    
    prev_p1=hidden_p1
    prev_p2=hidden_p2
    prev_p3=hidden_p3
    prev_p4=obs_p4

In [62]:
# a vague indicator of randomness
exe/count

0.0

In [63]:
data_final=data.append(data_pred,ignore_index="True")

In [64]:
data_final.to_csv("model2_lowRand.csv",sep="\t",header=False,index=False)